![https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png](https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png)

In [622]:
import pandas as pd
from math import *
import numpy as np
from scipy import interpolate

# Импорт компосовской херни
import pythoncom
from win32com.client import Dispatch, gencache

import LDefin2D
import MiscellaneousHelpers as MH



In [623]:
#  Подключим константы API Компас
kompas6_constants = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
kompas6_constants_3d = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants

#  Подключим описание интерфейсов API5
kompas6_api5_module = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
kompas_object = kompas6_api5_module.KompasObject(
    Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(kompas6_api5_module.KompasObject.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iKompasObject = kompas_object

#  Подключим описание интерфейсов API7
kompas_api7_module = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
application = kompas_api7_module.IApplication(
    Dispatch("Kompas.Application.7")._oleobj_.QueryInterface(kompas_api7_module.IApplication.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iApplication = application

documents = application.Documents
#  Получим активный документ
kompas_document = application.ActiveDocument
kompas_document_2d = kompas_api7_module.IKompasDocument2D(kompas_document)
iDocument2D = kompas_object.ActiveDocument2D()

In [624]:
table_79 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan79')

table_79

,№№,X,Y,H
0,ПЗ41,1483.980000,2374.220000,139.190000
1,111,1647.090040,2398.757954,143.581531
2,112,1598.359459,2517.749739,139.698306
3,113,1523.163780,2676.684287,140.361244
4,ПЗ42,1329.770000,2570.860000,138.720000


In [625]:
table_10 = pd.read_excel('Геодезия.xlsm', sheet_name='pandasPlan10')

table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,136.500000
2,2,111.0,78.183333,135.490000
3,3,155.0,136.400000,135.900000
4,4,132.0,144.916667,135.860000
5,5,141.0,195.450000,139.270000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,140.631531
8,7,131.0,181.350000,140.921531
9,8,128.0,231.250000,139.531531


In [626]:
def add_layer(id: int, state: int = 3, name: str = None):
    obj = iDocument2D.ksLayer(id)
    iLayerParam = kompas6_api5_module.ksLayerParam(kompas_object.GetParamStruct(kompas6_constants.ko_LayerParam))
    iLayerParam.Init()
    if name:
        iLayerParam.name = name
    iLayerParam.state = state
    iDocument2D.ksSetObjParam(obj, iLayerParam, LDefin2D.ALLPARAM)


def add_view(name: str, scale: float = 1, x=0, y=0):
    iViewParam = kompas6_api5_module.ksViewParam(kompas_object.GetParamStruct(kompas6_constants.ko_ViewParam))
    iViewParam.Init()
    iViewParam.angle = 0
    iViewParam.color = 0
    iViewParam.name = name
    iViewParam.scale_ = scale
    iViewParam.state = 3
    iViewParam.x = x
    iViewParam.y = y
    iDocument2D.ksCreateSheetView(iViewParam, 0)


def add_text(text: str, x: float, y: float, angle: float = 0, font_size: float = 2.5, color = 0):
    iParagraphParam = kompas6_api5_module.ksParagraphParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_ParagraphParam))
    iParagraphParam.Init()
    iParagraphParam.x = x
    iParagraphParam.y = y
    iParagraphParam.ang = angle
    iParagraphParam.height = 3.55
    iParagraphParam.width = 4
    iParagraphParam.hFormat = 0
    iParagraphParam.vFormat = 0
    iParagraphParam.style = 1
    iDocument2D.ksParagraph(iParagraphParam)

    iTextLineParam = kompas6_api5_module.ksTextLineParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextLineParam))
    iTextLineParam.Init()
    iTextLineParam.style = 1
    iTextItemArray = kompas_object.GetDynamicArray(LDefin2D.TEXT_ITEM_ARR)
    iTextItemParam = kompas6_api5_module.ksTextItemParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextItemParam))
    iTextItemParam.Init()
    iTextItemParam.iSNumb = 0
    iTextItemParam.s = text
    iTextItemParam.type = 0
    iTextItemFont = kompas6_api5_module.ksTextItemFont(iTextItemParam.GetItemFont())
    iTextItemFont.Init()
    iTextItemFont.bitVector = 4096
    iTextItemFont.color = color
    iTextItemFont.fontName = "GOST type A"
    iTextItemFont.height = font_size
    iTextItemFont.ksu = 1
    iTextItemArray.ksAddArrayItem(-1, iTextItemParam)
    iTextLineParam.SetTextItemArr(iTextItemArray)

    iDocument2D.ksTextLine(iTextLineParam)
    obj = iDocument2D.ksEndObj()



In [627]:
ONE_TO_SCALE = 2000

add_view("План", 1/ONE_TO_SCALE)
add_layer(1, 3,"Основные Точки")

for i, row in table_79.iterrows():
    # X и Y перевёрнуты
    obj = iDocument2D.ksCircle(row["Y"] * 1000, row["X"] * 1000, 1000, 1)
    add_text(str(row["№№"]) + " (" + str("{:1.2f}".format(row["H"])).replace(".", ",") + ")", row["Y"] * 1000 - 15000, row["X"] * 1000 - 7000)


In [628]:
# https://stackoverflow.com/questions/7586063/how-to-calculate-the-angle-between-a-line-and-the-horizontal-axis
def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX))

In [629]:
table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,136.500000
2,2,111.0,78.183333,135.490000
3,3,155.0,136.400000,135.900000
4,4,132.0,144.916667,135.860000
5,5,141.0,195.450000,139.270000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,140.631531
8,7,131.0,181.350000,140.921531
9,8,128.0,231.250000,139.531531


In [630]:
add_layer(2, 3,"Побочные Точки")

extra_points = {}

for i, row in table_10.iterrows():
    if " - " in str(row["№№"]):
        origin_label, dest_label = row["№№"].split(" - ")

        if origin_label.isdigit():
            origin_label = int(origin_label)

        if dest_label.isdigit():
            dest_label = int(dest_label)

        origin = (table_79.loc[table_79['№№'] == origin_label]["Y"].values[0], table_79.loc[table_79['№№'] == origin_label]["X"].values[0])

        _dest = (table_79.loc[table_79['№№'] == dest_label]["Y"].values[0], table_79.loc[table_79['№№'] == dest_label]["X"].values[0])

        angle = np.rad2deg(getAngleBetweenPoints( origin[0], origin[1], _dest[0], _dest[1]))
        _i_am_shit_at_coding = list(origin)
        _i_am_shit_at_coding.append(table_79.loc[table_79['№№'] == origin_label]["H"].values[0])
        extra_points.update({str(origin_label): _i_am_shit_at_coding})
        continue
    else:
        if not origin:
            continue
    _d = row["Отсчет по дальномеру kl, м"]
    _alpha = row["Отсчет по гориз. Кругу"]
    xx = origin[0] + (_d * cos(np.radians(angle - _alpha)))
    yy = origin[1] + (_d * sin(np.radians(angle - _alpha)))
    # obj = iDocument2D.ksLineSeg(origin[0] * 1000, origin[1] * 1000, xx * 1000,yy * 1000, 6)

    add_text(str(row["№№"]) + " (" + str("{:1.3f}".format(row["Отметки реечн. точек Hр.т., м"])).replace(".", ",") + ")", xx * 1000 + 2000, yy * 1000 - 1000)

    obj = iDocument2D.ksCircle(xx * 1000, yy * 1000, 1000, 1)
    extra_points.update({str(row["№№"]): [xx, yy, row["Отметки реечн. точек Hр.т., м"]]})



In [631]:
interpolated_points = {}

In [632]:
# CON_P41 = "1,2,4,5,21".split(",")
# CON_1 = ["21"]
# CON_5 =["4"]

add_layer(3,2,"Точки интерполяции пашни")

CON_111 = "5,6,7,8,9,112,ПЗ41".split(",")

center_point = np.array(extra_points["111"])
for point in CON_111:
    point = np.array(extra_points[point])

    if center_point[2] > point[2]:
        _coordinate_dif = center_point[:2] - point[:2]
        xia = [center_point[2], point[2]]
        hight_dif = range(int(ceil(point[2])), int(round(center_point[2])))
    else:
        _coordinate_dif = point[:2] - center_point[:2]
        xia = [point[2], center_point[2]]
        hight_dif = range(int(ceil(center_point[2])), int(round(point[2])))

    length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
    yia = [0, length]

    _alpha = np.rad2deg(getAngleBetweenPoints( center_point[0], center_point[1], point[0], point[1]))

    x = [0, length]
    f = interpolate.interp1d(xia, yia)

    for i in hight_dif:
        _d = f(i)
        xx = center_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = center_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx  ,yy , 0)
        add_text(i, xx - 10000, yy - 8000, color=9410565)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})


In [633]:
x = [extra_points[p][0] for p in extra_points]
y = [extra_points[p][1] for p in extra_points]
centroid = (sum(x) / len(extra_points), sum(y) / len(extra_points))
obj = iDocument2D.ksPoint(*(cord * 1000 for cord in centroid), 0)

current_view = kompas_document_2d.ViewsAndLayersManager.Views.ActiveView
current_view.X = 594/2 - centroid[0] / 2
current_view.Y = 420/2 - centroid[1] / 2
current_view.Update()


True

In [634]:
def interpolate_line(first_point: str, second_point: str, interpolated_points: dict):

    _first_point = np.array(extra_points[str(first_point)])
    _second_point = np.array(extra_points[str(second_point)])

    _coordinate_dif = _first_point[:2] - _second_point[:2]

    _length = round(np.sqrt(np.sum(np.power(_coordinate_dif, 2))))

    _yia = [0, _length]
    _xia = [ _first_point[2], _second_point[2]]

    _x = [0, _length]
    _f = interpolate.interp1d(_xia, _yia)

    _alpha = np.rad2deg(getAngleBetweenPoints(_first_point[0], _first_point[1], _second_point[0], _second_point[1]))

    # if np.diff(sorted([int(round(_second_point[2])), int(ceil(_first_point[2]))])) <= 1:
    #     return

    _hight_dif_sorted = sorted([_second_point[2], _first_point[2]])

    _height_dif = range(int(ceil(_hight_dif_sorted[0])), int(_hight_dif_sorted[1]) + 1)

    print(f"Points: {first_point, second_point}\n Range: {_height_dif}\n {_hight_dif_sorted}\n")

    for i in _height_dif:
        _d = _f(i)
        xx = _first_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = _first_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx, yy, 0)
        add_text(f"{i}", xx - 10000, yy - 8000, color=14417715)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})
        print(f"{i}: {xx}, {yy}")
    print("\n")

    return interpolated_points

In [635]:
# Интерполяция между 7 и 8 точками

# _seventh = np.array(extra_points["7"])
# _eighth = np.array(extra_points["8"])
#
# _coordinate_dif = _seventh[:2] - _eighth[:2]
#
# _length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
#
# _yia = [0, _length]
# _xia = [_seventh[2], _eighth[2]]
#
# _x = [0, _length]
# _f = interpolate.interp1d(_xia, _yia)
#
# _alpha = np.rad2deg(getAngleBetweenPoints( _seventh[0], _seventh[1], _eighth[0], _eighth[1]))
#
# _height_dif = range(int(round(_eighth[2])), int(ceil(_seventh[2])))
#
# print(_height_dif)
#
# for i in _height_dif:
#     print(i)
#     _d = _f(i)
#     xx = _seventh[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
#     yy = _seventh[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
#     obj = iDocument2D.ksPoint(xx  ,yy , 0)
#     add_text(f"{i}", xx - 10000, yy - 8000, color=16711680)
#
#     if i in interpolated_points:
#         interpolated_points[i].append((xx, yy))
#     else:
#         interpolated_points.update({i: [(xx, yy)]})

interpolate_line("7", "8", interpolated_points)


Points: ('7', '8')
 Range: range(140, 141)
 [139.5315311566677, 140.92153115666773]

140: 2479546.013409982, 1733374.8863937943




{140: [(2308910.7445719778, 1605869.432195151),
  (2496594.013874557, 1704019.8498474883),
  (2424365.5289027803, 1543184.6059996379),
  (2508860.574506494, 1601999.8293501304),
  (2378739.305426916, 1514020.9755505659),
  (2479546.013409982, 1733374.8863937943)],
 141: [(2333997.001249747, 1617378.6460708834),
  (2339557.869922507, 1704202.9662037895),
  (2420627.862287284, 1772330.4370231838),
  (2469277.188995107, 1688124.4668178968),
  (2417215.6331417076, 1572196.0621995262),
  (2478118.8008843535, 1614589.4757771243),
  (2384328.716258478, 1551175.214592027)],
 142: [(2359083.2579275155, 1628887.8599466158),
  (2362490.029541096, 1682079.3029753075),
  (2412156.1815317655, 1723816.4392665166),
  (2441960.3641156573, 1672229.0837883055),
  (2410065.7373806345, 1601207.5183994146),
  (2447377.0272622127, 1627179.122204118),
  (2389918.12709004, 1588329.4536334882)],
 143: [(2384169.5146052847, 1640397.0738223484),
  (2385422.189159685, 1659955.6397468254),
  (2403684.5007762476, 16

In [636]:
add_layer(4, 3,"Кривые интерполяции пашни")

_max_num_of_points = max([len(interpolated_points[height_points]) for height_points in interpolated_points])

for height in interpolated_points:

    if len(interpolated_points[height]) <= 3:
        continue

    cent=(sum([p[0] for p in interpolated_points[height]])/len(interpolated_points[height]),sum([p[1] for p in interpolated_points[height]])/len(interpolated_points[height]))
    interpolated_points[height].sort( key=lambda p: atan2(p[1]-cent[1],p[0]-cent[0]))

    obj = iDocument2D.ksBezier(len(interpolated_points[height]) == _max_num_of_points, 1)
    for point in interpolated_points[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [637]:
add_layer(5,2,"Точки интерполяции берега")

interpolated_points_shore = {}

# ПЗ 41
interpolate_line("4", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "1", interpolated_points_shore)
interpolate_line("ПЗ41", "2", interpolated_points_shore)
interpolate_line("ПЗ41", "4", interpolated_points_shore)
interpolate_line("ПЗ41", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "21", interpolated_points_shore)

# 112
# interpolate_line("8", "10", interpolated_points_shore)
# interpolate_line("112", "10", interpolated_points_shore)
# interpolate_line("112", "13", interpolated_points_shore)
# interpolate_line("9", "12", interpolated_points_shore)
interpolate_line("12", "13", interpolated_points_shore)
# interpolate_line("13", "1", interpolated_points_shore)
interpolate_line("1", "112", interpolated_points_shore)

# 113
interpolate_line("10", "14", interpolated_points_shore)
# interpolate_line("113", "15", interpolated_points_shore)
# interpolate_line("17", "113", interpolated_points_shore)
interpolate_line("16", "113", interpolated_points_shore)
# interpolate_line("17", "112", interpolated_points_shore)
# interpolate_line("17", "14", interpolated_points_shore)

# ПЗ 42
interpolate_line("ПЗ42", "20", interpolated_points_shore)
interpolate_line("ПЗ42", "18", interpolated_points_shore)
interpolate_line("23", "22", interpolated_points_shore)
interpolate_line("22", "21", interpolated_points_shore)
interpolate_line("20", "21", interpolated_points_shore)
interpolate_line("1", "21", interpolated_points_shore)
interpolate_line("21", "21", interpolated_points_shore)


Points: ('4', '5')
 Range: range(136, 140)
 [135.85999999999999, 139.27]

136: 2244379.581599439, 1495315.0881497024
137: 2258494.779658577, 1526588.0100096643
138: 2272609.977717715, 1557860.9318696263
139: 2286725.1757768528, 1589133.8537295884


Points: ('ПЗ41', '1')
 Range: range(137, 140)
 [136.5, 139.19]

137: 2420139.2208150467, 1428987.6170129855
138: 2399171.5400775825, 1454098.294176006
139: 2378203.8593401182, 1479208.9713390262


Points: ('ПЗ41', '2')
 Range: range(136, 140)
 [135.49, 139.19]

136: 2331836.4131267527, 1398177.1937302868
137: 2345122.8039961094, 1425074.6251627987
138: 2358409.1948654656, 1451972.0565953108
139: 2371695.5857348223, 1478869.4880278227


Points: ('ПЗ41', '4')
 Range: range(136, 140)
 [135.85999999999999, 139.19]

136: 2247945.2906453456, 1490644.3976861532
137: 2287529.8390323846, 1488555.2448064813
138: 2327114.3874194236, 1486466.0919268094
139: 2366698.9358064625, 1484376.9390471377


Points: ('ПЗ41', '5')
 Range: range(140, 140)
 [139.19, 

{136: [(2244379.581599439, 1495315.0881497024),
  (2331836.4131267527, 1398177.1937302868),
  (2247945.2906453456, 1490644.3976861532),
  (2415136.8293181052, 1343077.1870357494),
  (2540497.049406529, 1270784.5696018613),
  (2442941.486858289, 1326962.0748356963),
  (2427731.311840198, 1372884.1161330617)],
 137: [(2258494.779658577, 1526588.0100096643),
  (2420139.2208150467, 1428987.6170129855),
  (2345122.8039961094, 1425074.6251627987),
  (2287529.8390323846, 1488555.2448064813),
  (2402310.2370553766, 1387247.34783959),
  (2444263.1233333847, 1444913.7734495152),
  (2551659.898889423, 1292470.3896011768),
  (2470916.0904758475, 1346866.8313079134)],
 138: [(2272609.977717715, 1557860.9318696263),
  (2399171.5400775825, 1454098.294176006),
  (2358409.1948654656, 1451972.0565953108),
  (2327114.3874194236, 1486466.0919268094),
  (2389483.6447926476, 1431417.5086434302),
  (2461032.5567441615, 1501070.6347505453),
  (2471543.2476325976, 1501876.7634855951),
  (2562822.7483723164, 13

In [638]:
add_layer(6, 3,"Кривые интерполяции берега")

for height in interpolated_points_shore:

    if len(interpolated_points_shore[height]) < 3:
        continue

    interpolated_points_shore[height].sort()

    obj = iDocument2D.ksBezier(False, 1)
    for point in interpolated_points_shore[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [639]:
# Горизонталь вокруг ФС
add_layer(7,2,"Точки интерполяции ФС")

interpolated_points_fruit_garden = {}

_interpolation_21_23 = interpolate_line("21", "23", {})
_last_point_between_21_23 = list(_interpolation_21_23.keys())[-1]

interpolated_points_fruit_garden.update({_last_point_between_21_23: _interpolation_21_23[_last_point_between_21_23]})

_interpolation_112_17 = interpolate_line("112", "17", {})
if _last_point_between_21_23 in _interpolation_112_17:
    interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_112_17[_last_point_between_21_23])

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_14_17[_last_point_between_21_23])

_interpolation_15_113 = interpolate_line("15", "113", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_15_113[_last_point_between_21_23])

_interpolation_p42_18 = interpolate_line("ПЗ42", "18", {})
interpolated_points_fruit_garden[_last_point_between_21_23].insert(0, *_interpolation_p42_18[_last_point_between_21_23])


Points: ('21', '23')
 Range: range(136, 141)
 [135.32, 140.42]

136: 2444194.6752024475, 1328321.3110647516
137: 2474221.5257896753, 1350373.8837926658
138: 2504248.3763769036, 1372426.45652058
139: 2534275.2269641315, 1394479.029248494
140: 2564302.0775513593, 1416531.6019764082


Points: ('112', '17')
 Range: range(140, 141)
 [139.69830617334068, 140.76124394699454]

140: 2540722.333193306, 1570214.3349749742


Points: ('14', '17')
 Range: range(139, 141)
 [138.67124394699454, 140.76124394699454]

139: 2635219.4763654657, 1610827.0360958457
140: 2614501.2707160986, 1547125.9158880056


Points: ('15', '113')
 Range: range(140, 141)
 [139.00124394699452, 140.36124394699453]

140: 2689722.397648988, 1540611.2617111553


Points: ('ПЗ42', '18')
 Range: range(139, 141)
 [138.72, 140.42]

139: 2576769.3663110696, 1336417.1586803175
140: 2597874.2459934615, 1360157.0111100222




In [640]:
add_layer(8, 3,"Кривые интерполяции ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_fruit_garden[_last_point_between_21_23]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

1073760177

In [641]:
# Горизонталь за ФС
add_layer(9,2,"Точки интерполяции за ФС")

interpolated_points_behind_fruit_garden = {}

_interpolation_112_10 = interpolate_line("112", "10", {})
_last_point_between_112_10 = list(_interpolation_112_10.keys())[-1]

interpolated_points_behind_fruit_garden.update({_last_point_between_112_10: _interpolation_112_10[_last_point_between_112_10]})

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_14_17[_last_point_between_112_10])

_interpolation_15_113 = interpolate_line("15", "113", {})
if _last_point_between_112_10 in _interpolation_15_113:
    interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_15_113[_last_point_between_112_10])

_interpolation_8_10 = interpolate_line("8", "10", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].insert(0, *_interpolation_8_10[_last_point_between_112_10])


Points: ('112', '10')
 Range: range(139, 140)
 [138.88830617334068, 139.69830617334068]

139: 2562402.5071045454, 1658622.3720130017


Points: ('14', '17')
 Range: range(139, 141)
 [138.67124394699454, 140.76124394699454]

139: 2635219.4763654657, 1610827.0360958457
140: 2614501.2707160986, 1547125.9158880056


Points: ('15', '113')
 Range: range(140, 141)
 [139.00124394699452, 140.36124394699453]

140: 2689722.397648988, 1540611.2617111553


Points: ('8', '10')
 Range: range(139, 140)
 [138.88830617334068, 139.5315311566677]

139: 2559057.877631364, 1675793.4970725272




In [642]:
add_layer(10, 3,"Кривые интерполяции за ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_behind_fruit_garden[_last_point_between_112_10]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

1073760190

In [643]:
# Соединить основные границы

add_layer(10, 3,"Левая граница")

_left_border = "4,5,6,7".split(sep=",")

for i in range(len(_left_border)-1):
    # print(f"{ _left_border[i]}: {extra_points[ _left_border[i]]}\t{ _left_border[i+1]}: {extra_points[ _left_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _left_border[i]][:2], *extra_points[ _left_border[i+1]][:2])], 4)

In [644]:
add_layer(11, 3,"Верхняя граница")

_top_border = "7,8,10,14,15".split(sep=",")

for i in range(len(_top_border)-1):
    # print(f"{ _top_border[i]}: {extra_points[ _top_border[i]]}\t{ _top_border[i+1]}: {extra_points[ _top_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _top_border[i]][:2], *extra_points[ _top_border[i+1]][:2])], 1)

In [645]:
add_layer(12, 3,"Граница пашни")

_p_border = "5,ПЗ41,9,112,8".split(sep=",")

for i in range(len(_p_border)-1):
    # print(f"{ _p_border[i]}: {extra_points[ _p_border[i]]}\t{ _p_border[i+1]}: {extra_points[ _p_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _p_border[i]][:2], *extra_points[ _p_border[i+1]][:2])], 4)

In [646]:
add_layer(13, 3,"Река")

_river_border = "4,2,21,20".split(sep=",")

for i in range(len(_river_border)-1):
    # print(f"{ _river_border[i]}: {extra_points[ _river_border[i]]}\t{ _river_border[i+1]}: {extra_points[ _river_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2])], 1)

# вторая сторона реки

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["4"][:2], *extra_points["3"][:2]))

_second_shore = []
_d = 32
for i in range(len(_river_border)):
    # obj = iDocument2D.ksLineSeg(*[i * 1000 for i in tuple(map(lambda i, j: i - j, (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2]), (20,20,20,20)))], 1)

    xx = extra_points[ _river_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _river_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _second_shore.append((xx, yy))

for i in range(len(_second_shore) - 1):
    if i != len(_second_shore) - 2:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *_second_shore[i+1], 1)
    else:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *(p * 1000 for p in extra_points["19"][:2]), 1)

# Добавить подпись

_xx = (extra_points["2"][0] + extra_points["21"][0])/2 * 1000 + (_d/2 * 1000 * cos(np.radians(_alpha)))
_yy = (extra_points["2"][1] + extra_points["21"][1])/2 * 1000 + (_d/2 * 1000 * sin(np.radians(_alpha)))

add_text("р. Соть",_xx, _yy, np.rad2deg(getAngleBetweenPoints(*extra_points["2"][:2], *extra_points["21"][:2])), 5)

In [647]:
# Автодорога

add_layer(14, 3,"Автодорога")

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["6"][:2], *extra_points["7"][:2]))

_autobahn = []
_d = 10
for i in range(len(_top_border)):

    xx = extra_points[ _top_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _top_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _autobahn.append((xx, yy))

for i in range(len(_autobahn) - 1):
    obj = iDocument2D.ksLineSeg(*_autobahn[i], *_autobahn[i+1], 4)


In [648]:
from typing import List, Set, Dict, Tuple
from typing import Union

In [649]:
def endpoint_by_distance_and_angle(starting_point: Union[Tuple[float, float], List[float]], distance: float, angle: float) -> Tuple[float, float]:
    xx = starting_point[0] + (distance * cos(np.radians(angle)))
    yy = starting_point[1] + (distance * sin(np.radians(angle)))
    return xx, yy

In [650]:
# Текст пашни
add_layer(15, 3, "Текст пашни")

add_text("Пашня", *map(lambda i, j: (i + j) * 1000, extra_points["111"][:2], (-20, 45)),0, 7)

In [651]:
# Текст лес
add_layer(16, 3, "Текст лес")

add_text("Лес", *map(lambda i, j: (i + j) * 1000, extra_points["6"][:2], (-60, 0)),0, 7)

In [652]:
def m_to_mm(m: Tuple[float, float]) -> Tuple[float, float]:
    return tuple(map(lambda i: i * 1000, m))
def mm_to_m(m: Tuple[float, float]) -> Tuple[float, float]:
    return tuple(map(lambda i: i / 1000, m))

In [653]:
# https://stackoverflow.com/a/67313571
def get_intersections(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=sqrt((x1-x0)**2 + (y1-y0)**2)

    # non-intersecting
    if d > r0 + r1 :
        return {}
    # One circle within other
    if d < abs(r0-r1):
        return {}
    # coincident circles
    if d == 0 and r0 == r1:
        return {}
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d
        y2=y0+a*(y1-y0)/d
        x3=x2+h*(y1-y0)/d
        y3=y2-h*(x1-x0)/d
        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d
        return x3, y3, x4, y4


In [654]:
def add_rect(XY: Tuple[float, float], h: float, w: float, angle: float = 0, style: int = 1) -> int:
    iRectangleParam = kompas6_api5_module.ksRectangleParam(kompas_object.GetParamStruct(kompas6_constants.ko_RectangleParam))
    iRectangleParam.Init()
    iRectangleParam.x = XY[0]
    iRectangleParam.y = XY[1]
    iRectangleParam.ang = angle
    iRectangleParam.height = h
    iRectangleParam.width = w
    iRectangleParam.style = style
    return iDocument2D.ksRectangle(iRectangleParam)

In [655]:
# Фруктовый сад
add_layer(17, 3, "Колодец")

_alpha_14_15 = np.rad2deg(getAngleBetweenPoints(*extra_points["14"][:2], *extra_points["15"][:2]))
_alpha_113_P42 = np.rad2deg(getAngleBetweenPoints(*extra_points["113"][:2], *extra_points["ПЗ42"][:2]))


# _well_angles = degrees(atan2(14.62,9.15))
#
_well_point_1 = m_to_mm(endpoint_by_distance_and_angle(extra_points["113"][:2], 40, _alpha_113_P42))
# _obj_well_point_1 = iDocument2D.ksPoint(*_well_point_1, 0)

_well_point_2 = m_to_mm(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42))
# _obj_well_point_2 = iDocument2D.ksPoint(*_well_point_2, 0)
#
# well_line_1 = iDocument2D.ksLineSeg(*_well_point_1, *m_to_mm(endpoint_by_distance_and_angle(mm_to_m(_well_point_1), 9.15, _alpha_113_P42 - _well_angles)), 6)

_well_point_3 = get_intersections(*_well_point_1, 9.15*1000, *_well_point_2, 14.62*1000)

# Штриховка
_well_hatch = iDocument2D.ksHatch(0, 45, 0.25, 0, 0 ,0)
_obj_well_circle = iDocument2D.ksCircle(*_well_point_3[:2], 2500 ,1)
iDocument2D.ksEndObj()

_obj_well_circle = iDocument2D.ksCircle(*_well_point_3[:2], 2500 ,1)

add_text("Колодец", *map(lambda i, j: (i + j) * 1000, mm_to_m(_well_point_3[:2]), (-10, 5)))

add_layer(18, 3, "Фруктовый сад")

# fruit_garden_points = [endpoint_by_distance_and_angle(extra_points["113"][:2], 81.5, _alpha_113_P42 - (44 + (1/60))), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42), -8.2, _alpha_14_15), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 133.41, _alpha_113_P42), -7.81, _alpha_14_15), endpoint_by_distance_and_angle(extra_points["ПЗ42"][:2], 96.15, 0 - _alpha_113_P42 - (25 + (11/60)))]

fruit_garden_points = [tuple(extra_points["17"][:2]), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42), -8.2, _alpha_14_15), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 133.41, _alpha_113_P42), -7.81, _alpha_14_15), tuple(extra_points["23"][:2])]

iDocument2D.ksLineSeg(*m_to_mm(fruit_garden_points[0]), *m_to_mm(fruit_garden_points[3]), 1)
for i in range(len(fruit_garden_points)-1):
    iDocument2D.ksLineSeg(*m_to_mm(fruit_garden_points[i]), *m_to_mm(fruit_garden_points[i+1]), 1)

# _mid_fs_text = tuple(i/2 for i in (fruit_garden_points[0][0] + fruit_garden_points[1][0], fruit_garden_points[0][1] + fruit_garden_points[1][1]))
# *map(lambda i, j: (i + j) * 1000, _mid_fs_text, (-60, 0))

_alpha_fs = np.rad2deg(getAngleBetweenPoints(*fruit_garden_points[1], *fruit_garden_points[2]))

add_text("ФС", *m_to_mm(endpoint_by_distance_and_angle(fruit_garden_points[0], 15, _alpha_fs + 30)), np.rad2deg(getAngleBetweenPoints(*fruit_garden_points[0], *fruit_garden_points[1])), 5)

add_layer(19, 3, "2КЖ")

_kg2_p1 = m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 80.05, _alpha_113_P42), -8.03, _alpha_14_15))
_kg2_p2 = m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 110.23, _alpha_113_P42), -7.91, _alpha_14_15))

add_rect(_kg2_p1, -16.05*1000, (110.23-80.05)*1000, _alpha_fs)

# *tuple(i/2 for i in (_kg2_p1[0] + _kg2_p2[0], _kg2_p1[1] + _kg2_p2[1]))

add_text("2кж", _kg2_p2[0] - 2400, _kg2_p2[1] + 4000, np.rad2deg(getAngleBetweenPoints(*_kg2_p2, *_kg2_p1)),5)

In [656]:
def line_len(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    return sum(map(lambda fp, sp: (sp - fp) ** 2, p1, p2))**0.5



In [657]:
# Железная дорога
add_layer(19, 3, "Железная дорога")

_railroad_p1 = endpoint_by_distance_and_angle(extra_points["113"][:2], 12.64, _alpha_14_15)
_railroad_end = endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 162.1, _alpha_113_P42), 14.28, _alpha_14_15)

_alpha_rail = np.rad2deg(getAngleBetweenPoints(*_railroad_p1, *_railroad_end))

_railroad_p0 = endpoint_by_distance_and_angle(_railroad_p1, -70,_alpha_rail)

iDocument2D.ksLineSeg(*m_to_mm(_railroad_p0), *m_to_mm(_railroad_end), 1)

for i in range(5, int(line_len(_railroad_p0, _railroad_end))+5, 5):
    _start_point = endpoint_by_distance_and_angle(_railroad_p0, i,_alpha_rail)
    _side_point_1 = endpoint_by_distance_and_angle(_start_point, 2.5,  45 - _alpha_rail)
    _side_point_2 = endpoint_by_distance_and_angle(_start_point, -2.5, 45 - _alpha_rail)
    iDocument2D.ksLineSeg(*m_to_mm(_side_point_1), *m_to_mm(_side_point_2), 1)

In [658]:
# Условные обозначения леса, луга, фруктового сада.

def draw_meadow(point: Tuple[float, float], style: int = 2, size: float = 1000) -> Tuple[int, int]:

    line_one = iDocument2D.ksLineSeg(*point, *map(lambda i, j: (i + j), point, (0, 2.5 * size)), style)
    line_two = iDocument2D.ksLineSeg(*map(lambda i, j: (i + j), point, (1 * size, 0)), *map(lambda i, j: (i + j), point, (1 * size, 2.5 * size)), style)

    return line_one, line_two



In [674]:
# Добавляю этот пакет, что бы проще проверять находятся ли точки в полигонах
# https://stackoverflow.com/questions/52281055/point-in-polygon-using-shapely
from shapely import geometry, affinity

_plan_poly = [m_to_mm(tuple(extra_points["20"][:2])), m_to_mm(tuple(extra_points["21"][:2])), m_to_mm(tuple(extra_points["2"][:2])),
              m_to_mm(tuple(extra_points["4"][:2])), m_to_mm(tuple(extra_points["5"][:2])), m_to_mm(tuple(extra_points["6"][:2])),
              m_to_mm(tuple(extra_points["7"][:2])), m_to_mm(tuple(extra_points["8"][:2])), m_to_mm(tuple(extra_points["10"][:2])),
              m_to_mm(tuple(extra_points["14"][:2])), m_to_mm(tuple(extra_points["15"][:2])), m_to_mm(tuple(extra_points["113"][:2])),
              m_to_mm(tuple(extra_points["ПЗ42"][:2]))]

_plan_poly_line = geometry.LineString(_plan_poly)
_plan_poly = geometry.Polygon(_plan_poly_line)

_farm_poly = [m_to_mm(tuple(extra_points["ПЗ41"][:2])), m_to_mm(tuple(extra_points["5"][:2])), m_to_mm(tuple(extra_points["6"][:2])),
              m_to_mm(tuple(extra_points["7"][:2])), m_to_mm(tuple(extra_points["8"][:2])), m_to_mm(tuple(extra_points["112"][:2])),
              m_to_mm(tuple(extra_points["9"][:2]))]

_farm_poly_line = geometry.LineString(_farm_poly)
_farm_poly = geometry.Polygon(_farm_poly_line)

_fruit_garden_poly = geometry.Polygon(geometry.LineString((m_to_mm(i) for i in fruit_garden_points)))

_fruit_garden_poly = affinity.scale(_fruit_garden_poly, 0.95, 0.95)

_kg_poly = [_kg2_p1, m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 80.05, _alpha_113_P42), -8.03 - 16.05, _alpha_14_15)), m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 110.23, _alpha_113_P42), -7.91 -16.05, _alpha_14_15)), _kg2_p2]

_kg_poly_line = geometry.LineString(_kg_poly)
_kg_poly = geometry.Polygon(_kg_poly_line)


In [677]:
add_layer(20, 0, "Условные обозначения луга")

for i in range(int(extra_points["20"][1] - int(extra_points["20"][1]) % 5), int(extra_points["7"][1]), 20):
    for b in range(int(extra_points["4"][0] - int(extra_points["4"][1]) % 5), int(extra_points["15"][0]), 20):
        _d_point = geometry.Point(m_to_mm((b, i)))
        if _plan_poly.contains(_d_point) and not _farm_poly.contains(_d_point) and not _fruit_garden_poly.contains(_d_point):
            draw_meadow(m_to_mm((b, i)))


(2522, 1265)
(2482, 1285)
(2502, 1285)
(2522, 1285)
(2542, 1285)
(2442, 1305)
(2462, 1305)
(2482, 1305)
(2502, 1305)
(2522, 1305)
(2542, 1305)
(2422, 1325)
(2442, 1325)
(2462, 1325)
(2482, 1325)
(2502, 1325)
(2522, 1325)
(2542, 1325)
(2562, 1325)
(2382, 1345)
(2402, 1345)
(2422, 1345)
(2442, 1345)
(2462, 1345)
(2482, 1345)
(2502, 1345)
(2522, 1345)
(2542, 1345)
(2562, 1345)
(2362, 1365)
(2382, 1365)
(2402, 1365)
(2422, 1365)
(2442, 1365)
(2462, 1365)
(2482, 1365)
(2502, 1365)
(2522, 1365)
(2542, 1365)
(2562, 1365)
(2582, 1365)
(2342, 1385)
(2362, 1385)
(2382, 1385)
(2402, 1385)
(2422, 1385)
(2442, 1385)
(2462, 1385)
(2482, 1385)
(2502, 1385)
(2522, 1385)
(2542, 1385)
(2562, 1385)
(2582, 1385)
(2322, 1405)
(2342, 1405)
(2362, 1405)
(2382, 1405)
(2402, 1405)
(2422, 1405)
(2442, 1405)
(2462, 1405)
(2482, 1405)
(2502, 1405)
(2522, 1405)
(2542, 1405)
(2562, 1405)
(2582, 1405)
(2602, 1405)
(2302, 1425)
(2322, 1425)
(2342, 1425)
(2362, 1425)
(2382, 1425)
(2402, 1425)
(2422, 1425)
(2442, 1425)

In [678]:
add_layer(21, 0, "Условные обозначения фруктового сада")

for i in range(int(fruit_garden_points[2][1]), int(fruit_garden_points[0][1]), 5):
    for b in range(int(fruit_garden_points[-1][0]), int(fruit_garden_points[1][0]), 5):
        _d_point = geometry.Point(m_to_mm((b, i)))
        if _fruit_garden_poly.contains(_d_point) and not _kg_poly.contains(_d_point):
            iDocument2D.ksCircle(*m_to_mm((b, i)), 1000 ,2)